# Evaluation Notebook

This notebook retrieves tasks from Notion that match debug tasks in an eval folder, and prepares them for comparison and evaluation.

In [ ]:
# Imports
import json
import re
from pathlib import Path
from typing import Optional
from notion_client import Client

from inbox_agent.config import settings
from inbox_agent.notion import query_pages_filtered, get_block_plain_text, extract_property_value
from inbox_agent.pydantic_models import NotionTask, AIUseStatus

print("✓ Imports loaded")

In [ ]:
# ============ CONFIGURATION ============
# Control which eval folder to use
EVAL_FOLDER_NAME = "eval_1"  # Change this to use different eval folders (e.g., "eval_2", "eval_3")

# File names for processed tasks
NOTION_TASKS_FILE_NAME = "notion_tasks.json"
DEBUG_TASKS_FILE_NAME = "debug_tasks.json"

# Derived paths
EVAL_FOLDER_PATH = settings.LOGS_DIR / EVAL_FOLDER_NAME
NOTION_TASKS_FILE = EVAL_FOLDER_PATH / NOTION_TASKS_FILE_NAME
DEBUG_TASKS_FILE = EVAL_FOLDER_PATH / DEBUG_TASKS_FILE_NAME

print(f"✓ Eval folder: {EVAL_FOLDER_PATH}")
print(f"✓ Notion tasks will be saved to: {NOTION_TASKS_FILE}")
print(f"✓ Debug tasks will be saved to: {DEBUG_TASKS_FILE}")

In [ ]:
# ============ STEP 1: Extract Debug Task Information ============

def get_debug_task_files(eval_folder: Path) -> list[Path]:
    """Get all .md debug task files from eval folder."""
    return [f for f in eval_folder.glob("*.md") if f.is_file()]

def extract_task_model_from_debug_file(file_path: Path) -> dict:
    """
    Extract the Task Model JSON from a debug task markdown file.
    
    Returns:
        Dict with task model data, or None if not found
    """
    content = file_path.read_text(encoding='utf-8')
    
    # Find Task Model section
    match = re.search(r'## Task Model\s+```json\s+(.*?)\s+```', content, re.DOTALL)
    if not match:
        return None
    
    try:
        task_data = json.loads(match.group(1))
        return task_data
    except json.JSONDecodeError as e:
        print(f"Warning: Could not parse Task Model JSON in {file_path.name}: {e}")
        return None

def get_debug_task_titles(eval_folder: Path) -> list[str]:
    """Extract all task titles from debug task files in eval folder."""
    debug_files = get_debug_task_files(eval_folder)
    titles = []
    
    for file in debug_files:
        task_model = extract_task_model_from_debug_file(file)
        if task_model and 'title' in task_model:
            titles.append(task_model['title'])
    
    return titles

def load_and_save_debug_tasks(eval_folder: Path, output_file: Path) -> list[dict]:
    """
    Load debug tasks from markdown files and save to JSON.
    
    Args:
        eval_folder: Folder containing debug task markdown files
        output_file: Path to save the JSON file
    
    Returns:
        List of debug task model dicts
    """
    debug_files = get_debug_task_files(eval_folder)
    debug_tasks = []
    
    for file in debug_files:
        task_model = extract_task_model_from_debug_file(file)
        if task_model:
            debug_tasks.append(task_model)
    
    # Save as JSON
    output_file.parent.mkdir(parents=True, exist_ok=True)
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(debug_tasks, f, indent=2, ensure_ascii=False)
    
    return debug_tasks

# Test: Get debug task titles
debug_titles = get_debug_task_titles(EVAL_FOLDER_PATH)
print(f"✓ Found {len(debug_titles)} debug tasks")
print(f"Sample titles: {debug_titles[:3]}")

# Load and save debug tasks to JSON
debug_tasks_loaded = load_and_save_debug_tasks(EVAL_FOLDER_PATH, DEBUG_TASKS_FILE)
print(f"✓ Loaded and saved {len(debug_tasks_loaded)} debug tasks to {DEBUG_TASKS_FILE}")

In [ ]:
# ============ STEP 2: Retrieve Tasks from Notion ============

def retrieve_tasks_from_notion(notion_client: Client, task_titles: list[str]) -> list[dict]:
    """
    Retrieve tasks from Notion based on titles using OR filter.
    
    Args:
        notion_client: Notion client instance
        task_titles: List of task titles to retrieve
    
    Returns:
        List of Notion page objects
    """
    if not task_titles:
        return []
    
    # Build OR filter for all titles
    filter_dict = {
        "or": [
            {"property": "Name", "title": {"equals": title}}
            for title in task_titles
        ]
    }
    
    # Properties to retrieve (relevant to NotionTask model + AIUsefulness, UseAIStatus)
    filter_properties = [
        "Name",           # title
        "Project",        # projects (relation)
        "Importance",     # importance
        "Urgency",        # urgency
        "Impact Score",   # impact
        "UseAIStatus",    # ai_use_status
        "AIUsefulness"    # additional evaluation property
    ]
    
    # Query Notion
    response = query_pages_filtered(
        notion_client,
        settings.NOTION_TASKS_DATA_SOURCE_ID,
        filter_dict=filter_dict,
        filter_properties=filter_properties
    )
    
    return response['results']

# Initialize Notion client and retrieve tasks
notion = Client(auth=settings.NOTION_TOKEN)
notion_tasks_raw = retrieve_tasks_from_notion(notion, debug_titles)

print(f"✓ Retrieved {len(notion_tasks_raw)} tasks from Notion")
if notion_tasks_raw:
    print(f"Sample task ID: {notion_tasks_raw[0]['id']}")

In [ ]:
# ============ STEP 3: Parse and Save Notion Tasks ============

def parse_notion_task(notion_page: dict) -> dict:
    """
    Parse a Notion page object into a simplified task dict.
    
    Returns:
        Dict with task properties extracted from Notion
    """
    props = notion_page.get('properties', {})
    
    # Extract title
    title = get_block_plain_text(notion_page)
    
    # Extract properties using reusable helper
    importance = extract_property_value(props.get('Importance'))
    urgency = extract_property_value(props.get('Urgency'))
    ai_status = extract_property_value(props.get('UseAIStatus'))
    ai_usefulness = extract_property_value(props.get('AIUsefulness'))
    impact = extract_property_value(props.get('Impact Score'))
    
    # Convert string importance/urgency to int if available
    importance = int(importance) if importance else None
    urgency = int(urgency) if urgency else None
    
    # Extract project relations
    project_relations = props.get('Project', {}).get('relation', [])
    project_ids = [rel['id'] for rel in project_relations]
    
    return {
        'id': notion_page['id'],
        'title': title,
        'importance': importance,
        'urgency': urgency,
        'impact': impact,
        'ai_use_status': ai_status,
        'ai_usefulness': ai_usefulness,
        'project_ids': project_ids,
        'created_time': notion_page.get('created_time'),
        'last_edited_time': notion_page.get('last_edited_time')
    }

def save_notion_tasks(tasks_raw: list[dict], output_file: Path):
    """Parse and save Notion tasks to JSON file."""
    parsed_tasks = [parse_notion_task(task) for task in tasks_raw]
    
    # Save as JSON
    output_file.parent.mkdir(parents=True, exist_ok=True)
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(parsed_tasks, f, indent=2, ensure_ascii=False)
    
    return parsed_tasks

# Parse and save
notion_tasks_parsed = save_notion_tasks(notion_tasks_raw, NOTION_TASKS_FILE)
print(f"✓ Saved {len(notion_tasks_parsed)} parsed Notion tasks to {NOTION_TASKS_FILE}")
print(f"✓ Sample task: {notion_tasks_parsed[0]['title'] if notion_tasks_parsed else 'None'}")

In [ ]:
# ============ STEP 4: Load and Pair Tasks ============

def load_tasks_from_json(json_file: Path) -> list[dict]:
    """
    Generic loader for tasks from JSON file.
    Works for both Notion tasks and debug tasks.
    """
    if not json_file.exists():
        return []
    
    with open(json_file, 'r', encoding='utf-8') as f:
        return json.load(f)

def create_task_pairs(notion_tasks_file: Path, debug_tasks_file: Path) -> list[tuple[dict, dict]]:
    """
    Create pairs of (notion_task, debug_task) for evaluation by matching titles.
    
    Args:
        notion_tasks_file: Path to Notion tasks JSON file
        debug_tasks_file: Path to debug tasks JSON file
    
    Returns:
        List of tuples where each tuple is (notion_task_dict, debug_task_model_dict)
    """
    # Load both datasets from JSON files
    notion_tasks = load_tasks_from_json(notion_tasks_file)
    debug_tasks = load_tasks_from_json(debug_tasks_file)
    
    # Create dict for O(1) lookup by title
    debug_tasks_by_title = {task['title']: task for task in debug_tasks}
    
    # Create pairs based on matching titles
    pairs = []
    matched_titles = set()
    
    for notion_task in notion_tasks:
        title = notion_task['title']
        if title in debug_tasks_by_title:
            pairs.append((notion_task, debug_tasks_by_title[title]))
            matched_titles.add(title)
    
    # Report unmatched tasks
    unmatched_debug = set(debug_tasks_by_title.keys()) - matched_titles
    unmatched_notion = {t['title'] for t in notion_tasks} - matched_titles
    
    print(f"✓ Created {len(pairs)} task pairs")
    if unmatched_debug:
        print(f"⚠ {len(unmatched_debug)} debug tasks not found in Notion")
        print(f"  Sample unmatched: {list(unmatched_debug)[:3]}")
    if unmatched_notion:
        print(f"⚠ {len(unmatched_notion)} Notion tasks not found in debug")
        print(f"  Sample unmatched: {list(unmatched_notion)[:3]}")
    
    return pairs

# Create task pairs
task_pairs = create_task_pairs(NOTION_TASKS_FILE, DEBUG_TASKS_FILE)
print(f"\n{'='*60}")
print(f"EVALUATION DATASET READY")
print(f"{'='*60}")
print(f"Total pairs: {len(task_pairs)}")
if task_pairs:
    print(f"\nSample pair:")
    print(f"  Notion task: {task_pairs[0][0]['title']}")
    print(f"  Debug task:  {task_pairs[0][1]['title']}")

In [ ]:
# ============ HELPER FUNCTIONS FOR ANALYSIS ============

def inspect_pair(pair_index: int = 0):
    """Display detailed comparison of a task pair."""
    if pair_index >= len(task_pairs):
        print(f"Error: Only {len(task_pairs)} pairs available")
        return
    
    notion_task, debug_task = task_pairs[pair_index]
    
    print(f"{'='*70}")
    print(f"TASK PAIR #{pair_index}: {notion_task['title']}")
    print(f"{'='*70}\n")
    
    print("📊 NOTION TASK (Actual):")
    print(f"  Importance: {notion_task['importance']}")
    print(f"  Urgency:    {notion_task['urgency']}")
    print(f"  Impact:     {notion_task['impact']}")
    print(f"  AI Status:  {notion_task['ai_use_status']}")
    print(f"  AI Useful:  {notion_task['ai_usefulness']}")
    print(f"  Projects:   {len(notion_task['project_ids'])} linked")
    
    print("\n🔍 DEBUG TASK (Expected):")
    print(f"  Importance: {debug_task['importance']}")
    print(f"  Urgency:    {debug_task['urgency']}")
    print(f"  Impact:     {debug_task['impact']}")
    print(f"  AI Status:  {debug_task['ai_use_status']}")
    print(f"  Projects:   {len(debug_task['projects'])} in model")
    print(f"  Confidence: {debug_task.get('confidence', 'N/A')}")
    
    print("\n📝 Original Note:")
    print(f"  {debug_task['original_note'][:200]}...")
    
    if debug_task.get('enrichment'):
        print("\n✨ Enrichment:")
        print(f"  {debug_task['enrichment'][:200]}...")

# Example: Inspect first pair
if task_pairs:
    inspect_pair(0)

In [ ]:
# ============ QUICK ACCESS TO DATA ============

# The main data structure for evaluation:
# task_pairs = [(notion_task, debug_task), ...]
#
# Access pattern:
# for notion_task, debug_task in task_pairs:
#     # Compare notion_task['importance'] with debug_task['importance']
#     # etc.

print("\n📦 Available data structures:")
print(f"  task_pairs:           {len(task_pairs)} pairs of (notion_task, debug_task)")
print(f"  notion_tasks_parsed:  {len(notion_tasks_parsed)} Notion tasks")
print(f"  debug_tasks_loaded:   {len(debug_tasks_loaded)} debug tasks")
print(f"  debug_titles:         {len(debug_titles)} debug task titles")
print(f"\n💾 Files:")
print(f"  {NOTION_TASKS_FILE}")
print(f"  {DEBUG_TASKS_FILE}")
print(f"\n🔧 Helper functions:")
print(f"  inspect_pair(index) - Detailed view of a task pair")
print(f"  load_tasks_from_json(file) - Generic JSON loader for tasks")
print(f"  get_debug_task_files() - List all debug task markdown files")
print(f"  create_task_pairs(notion_file, debug_file) - Create pairs from JSON files")

print(f"\n{'='*70}")
print("Ready for evaluation! 🚀")
print(f"{'='*70}")

## Analysis & Metrics

This section computes evaluation metrics comparing debug tasks (expected) with Notion tasks (actual).

In [ ]:
# Imports for analysis
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

print("✓ Analysis libraries loaded")

In [ ]:
# ============ COMPUTE METRICS ============

def compute_metrics(pairs: list[tuple[dict, dict]]) -> dict:
    """
    Compute accuracy metrics for each property.
    
    Args:
        pairs: List of (notion_task, debug_task) tuples
    
    Returns:
        Dict with metric summaries
    """
    if not pairs:
        return {}
    
    n = len(pairs)
    
    # Initialize counters
    importance_correct = 0
    urgency_correct = 0
    ai_status_correct = 0
    impact_diffs = []
    
    for notion_task, debug_task in pairs:
        # Importance
        if notion_task['importance'] == debug_task['importance']:
            importance_correct += 1
        
        # Urgency
        if notion_task['urgency'] == debug_task['urgency']:
            urgency_correct += 1
        
        # AI Status
        if notion_task['ai_use_status'] == debug_task['ai_use_status']:
            ai_status_correct += 1
        
        # Impact (MAE)
        if notion_task['impact'] is not None and debug_task['impact'] is not None:
            diff = abs(notion_task['impact'] - debug_task['impact'])
            impact_diffs.append(diff)
    
    metrics = {
        'n_pairs': n,
        'importance_accuracy': importance_correct / n,
        'urgency_accuracy': urgency_correct / n,
        'ai_status_accuracy': ai_status_correct / n,
        'impact_mae': np.mean(impact_diffs) if impact_diffs else None,
        'impact_rmse': np.sqrt(np.mean([d**2 for d in impact_diffs])) if impact_diffs else None
    }
    
    return metrics

# Compute and display metrics
metrics = compute_metrics(task_pairs)

print("📊 EVALUATION METRICS")
print("="*60)
print(f"Sample size: {metrics['n_pairs']} task pairs")
print()
print(f"Importance Accuracy:  {metrics['importance_accuracy']:.2%}")
print(f"Urgency Accuracy:     {metrics['urgency_accuracy']:.2%}")
print(f"AI Status Accuracy:   {metrics['ai_status_accuracy']:.2%}")
print(f"Impact MAE:           {metrics['impact_mae']:.3f}" if metrics['impact_mae'] else "Impact MAE:           N/A")
print(f"Impact RMSE:          {metrics['impact_rmse']:.3f}" if metrics['impact_rmse'] else "Impact RMSE:          N/A")
print("="*60)

In [ ]:
# ============ DISTRIBUTION PLOTS ============

def plot_distributions(pairs: list[tuple[dict, dict]]):
    """Create distribution plots comparing predicted vs actual for each property."""
    
    # Extract data
    notion_importance = [n['importance'] for n, d in pairs if n['importance'] is not None]
    debug_importance = [d['importance'] for n, d in pairs if d['importance'] is not None]
    
    notion_urgency = [n['urgency'] for n, d in pairs if n['urgency'] is not None]
    debug_urgency = [d['urgency'] for n, d in pairs if d['urgency'] is not None]
    
    notion_ai_status = [n['ai_use_status'] for n, d in pairs if n['ai_use_status'] is not None]
    debug_ai_status = [d['ai_use_status'] for n, d in pairs if d['ai_use_status'] is not None]
    
    # Create figure
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    fig.suptitle('Property Distributions: Notion (Actual) vs Debug (Expected)', fontsize=14, fontweight='bold')
    
    # Importance distribution
    if notion_importance and debug_importance:
        counts_n = Counter(notion_importance)
        counts_d = Counter(debug_importance)
        labels = sorted(set(notion_importance + debug_importance))
        
        x = np.arange(len(labels))
        width = 0.35
        
        axes[0].bar(x - width/2, [counts_n.get(l, 0) for l in labels], width, label='Notion', alpha=0.8)
        axes[0].bar(x + width/2, [counts_d.get(l, 0) for l in labels], width, label='Debug', alpha=0.8)
        axes[0].set_xlabel('Importance Value')
        axes[0].set_ylabel('Count')
        axes[0].set_title('Importance')
        axes[0].set_xticks(x)
        axes[0].set_xticklabels(labels)
        axes[0].legend()
        axes[0].grid(axis='y', alpha=0.3)
    
    # Urgency distribution
    if notion_urgency and debug_urgency:
        counts_n = Counter(notion_urgency)
        counts_d = Counter(debug_urgency)
        labels = sorted(set(notion_urgency + debug_urgency))
        
        x = np.arange(len(labels))
        width = 0.35
        
        axes[1].bar(x - width/2, [counts_n.get(l, 0) for l in labels], width, label='Notion', alpha=0.8)
        axes[1].bar(x + width/2, [counts_d.get(l, 0) for l in labels], width, label='Debug', alpha=0.8)
        axes[1].set_xlabel('Urgency Value')
        axes[1].set_ylabel('Count')
        axes[1].set_title('Urgency')
        axes[1].set_xticks(x)
        axes[1].set_xticklabels(labels)
        axes[1].legend()
        axes[1].grid(axis='y', alpha=0.3)
    
    # AI Status distribution
    if notion_ai_status and debug_ai_status:
        counts_n = Counter(notion_ai_status)
        counts_d = Counter(debug_ai_status)
        labels = sorted(set(notion_ai_status + debug_ai_status))
        
        x = np.arange(len(labels))
        width = 0.35
        
        axes[2].bar(x - width/2, [counts_n.get(l, 0) for l in labels], width, label='Notion', alpha=0.8)
        axes[2].bar(x + width/2, [counts_d.get(l, 0) for l in labels], width, label='Debug', alpha=0.8)
        axes[2].set_xlabel('AI Use Status')
        axes[2].set_ylabel('Count')
        axes[2].set_title('AI Use Status')
        axes[2].set_xticks(x)
        axes[2].set_xticklabels([str(l) for l in labels], rotation=15, ha='right')
        axes[2].legend()
        axes[2].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Create distribution plots
plot_distributions(task_pairs)